In [1]:
import sncosmo
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sncosmo.salt2utils import BicubicInterpolator
from scipy.interpolate import (
    InterpolatedUnivariateSpline as Spline1d,
    RectBivariateSpline as Spline2d
)
import pickle as pk
from matplotlib.backends.backend_pdf import PdfPages
import os 
from matplotlib.offsetbox import AnchoredText
import astropy.constants as const
import astropy.units as u
import seaborn as sns
from matplotlib.ticker import ScalarFormatter

sns.set_style("whitegrid", {'axes.grid':'False','xtick.bottom': True,'ytick.left': True})

params = {"text.usetex" : True,'font.size': 20, 'font.family':'serif', 'font.serif':'Computer Modern'}
plt.rcParams.update(params)

H_ERG_S = const.h.cgs.value
C_AA_PER_S = const.c.to(u.AA / u.s).value
HC_ERG_AA = H_ERG_S * C_AA_PER_S
SCALE_FACTOR = 1e-12

In [2]:
class ScalarFormatterClass1(ScalarFormatter):
    def _set_format(self):
        self.format = "%1.1f"

## Comentários

In [3]:
#

## Recuperando curvas de luz

In [4]:
path="/home/cassia/SNANA/snroot/lcmerge/JLA2014_SNLS/"

In [5]:
sne_snls_pan=pd.read_csv("snls_pantheon.txt",header=None,dtype=str)

In [6]:
sne_list=np.array(sne_snls_pan).T[0]

In [7]:
path_filters="/home/cassia/SNANA/snroot/filters/PS1/Pantheon/SNLS3-Megacam/"

## Registrando filtros

In [8]:
ab = sncosmo.get_magsystem('ab')

In [9]:
for sn in sne_list:
    for filt in ["g","r","i","z"]:
        wave = pd.read_csv(path_filters+"effMEGACAM-"+filt+".dat",header=None,sep="\s+",comment="#")[0].values
        trans = pd.read_csv(path_filters+"effMEGACAM-"+filt+".dat",header=None,sep="\s+",comment="#")[1].values
        band = sncosmo.Bandpass(wave, trans, name="SNLS_"+filt,trim_level=0.05)
        
        sncosmo.registry.register(band, force=True)

In [10]:
magsystem={"SNLS_g":("ab",-0.0067+ 0.0076605430),
        "SNLS_r":("ab",-0.0081-0.0012669564),
        "SNLS_i":("ab",-0.0073-0.0057157707),
        "SNLS_z":("ab",-0.0078+0.0024809218)}

## Criando modelo EXP

In [11]:
class EXP_LC(sncosmo.Source):
    
    _param_names = ['x0', 'x1', 'x2']
    param_names_latex = ['x_0', 'x_1', 'x_2']
    
    
    def __init__(self, phase, wave, flux0, flux1, flux2, name='EXP', version='2021'):

        self.name = name
        self.version = version
        self._phase = phase
        self._wave = wave
        
        self._model_flux0  = BicubicInterpolator(phase, wave, flux0.T)
        self._model_flux1  = BicubicInterpolator(phase, wave, flux1.T)
        self._model_flux2  = BicubicInterpolator(phase, wave, flux2.T)

        self._parameters = np.array([1, 0, 0])  # initial guess
        

    def _flux(self, phase, wave):
        
        x0, x1, x2= self._parameters
        #print('flux!')
        return x0 * (self._model_flux0(phase, wave) +  x1 * self._model_flux1(phase, wave) +  x2 * self._model_flux2(phase, wave))

In [12]:
gridx2 = np.linspace(-10, 50, 50+10+1, dtype=int)
gridy2 = np.linspace(3400, 8400, int((8400-3400)/10+1), dtype=int)

In [13]:
X, Y = np.meshgrid(gridx2,gridy2)

#### Templates FA

In [14]:
M_fa=[]
for i in range(3):
    M_fa.append(np.loadtxt("../fa/sk_varimax_M"+str(i)+".txt")*SCALE_FACTOR)

#### Templates PCA

In [15]:
M_pca=[]
for i in range(3):
    M_pca.append(np.loadtxt("../pca/M"+str(i)+"_pca.txt")*SCALE_FACTOR)

### Criando os modelos

In [16]:
exp_fa = EXP_LC(gridx2, gridy2, M_fa[0], M_fa[1], M_fa[2])
exp_pca = EXP_LC(gridx2, gridy2, M_pca[0], M_pca[1], M_pca[2])

In [17]:
sncosmo.register(exp_fa,name="exp_fa_varimax")
sncosmo.register(exp_pca,name="exp_pca")

## Escolha do modelo

In [18]:
modelo="exp_pca"

## Ajustando curvas de luz

In [19]:
def lc_fit(sn):    
    try:
        meta, tables = sncosmo.read_snana_ascii(path+"JLA2014_SNLS_"+sn+".dat", default_tablename='OBS')
    except:
        print(f"File not found for SN {sn}")
        return None
    
    dat=tables["OBS"]
    surv="SNLS"

    dat.add_column([surv+"_"+f for f in dat["FLT"]], name='FILTER')
    dat.remove_column("FLT")
     
    dat.remove_rows(np.where(dat["FLUXCAL"]<0)[0])
    dat.remove_rows(np.where(np.isnan(dat["MAG"]))[0])
     
    magsys=sncosmo.CompositeMagSystem(bands=magsystem,name="magsys")
    sncosmo.register(magsys,force=True)
        
    dat["ZP"]=[2.5*np.log10(magsys.zpbandflux(f)) for f in dat["FILTER"]]    
    dat["FLUX"]=[10**(-0.4*dat["MAG"][i]+0.4*dat["ZP"][i]) for i in range(len(dat["MAG"]))]
    dat["FLUXERR"]=[abs(0.4*dat["FLUX"][i]*np.log(10.)*dat["MAGERR"][i]) for i in range(len(dat["MAG"]))]
    dat.add_column("ab", name='ZPSYS')

    dat.remove_columns(["FIELD","SNR","MAG","MAGERR","Zpt"])
    
    # ref de repouso: (t-t0)/(1+z), no SNANA fazem um corte de -15 a 45 no referencial de repouso. E também tem um corte no 
    # intervalo de comprimento de onda, eu apliquei esses cortes abaixo em phase_range e wave_range. Porém não tenho certeza
    # se o sncosmo corta no ref de repouso ou do observador. Alterando isso, os valores de x1 para 2001ah e 2001az mudam drasticamento
    # equivalendo a mesma mudança que ocorre no snana quando fazemos a mesma alteração no TREST_REJECT.    
        
    dust = sncosmo.F99Dust() 

    # para algumas sne como 2006bb o chi2 da minimizacao da Nan e não é possível ajustar a sn. A mesma sn no snana é ajustada pelo salt.
    # alterando intervalo de dias e comprimento de onda de atuação do ajustador, ainda não é possível evitar o nan já no primeiro passo.
    
    rm_f=[]
    if sn in ["03D1fq","03D4cx","03D4cz","03D4di","04D1iv","04D1pp","04D2iu","04D3dd","04D3lu","04D3ml","04D4dw","04D4ic","04D4ih",
              "04D4jw","05D1az","05D1cl","05D3kp","05D4cs","05D4ev","06D2ce","06D4ce","03D4cy","04D1de","04D1hy","04D2gp","04D3fq",
              "04D3lp","04D3nc","04D3nr","04D3oe","04D4bk","04D4dm","04D4hf","04D4ib","05D1eo","05D1iz","05D2by","05D2my","05D2nt",
              "05D3cx","05D3gv","05D3ht","05D3jh","05D4bi","05D2ob","05D2nn","05D4dw","04D4im","05D2bw","05D4hn","06D4cl","06D3bz","06D3en"]:
        dat.remove_rows(np.where(dat["FILTER"]=="SNLS_g"))
        rm_f.append("SNLS_g")
    if sn in ["03D1au","04D1iv","04D4an","04D4hu","04D4ic","04D4ih","04D4im","04D4jw","05D2hc","05D3gp","05D4bf","05D4ev","06D2cc",
              "06D2ce","06D3em","03D1aw","04D1jd","04D1pu","04D4id","04D4ii","05D1kl","05D3ci","05D3mh","05D3mq","05D4bi","06D2cd",
              "06D2ck","06D2ga","05D1hk","06D3gn"]:
        dat.remove_rows(np.where(dat["FILTER"]=="SNLS_z"))
        rm_f.append("SNLS_z")
    
    model = sncosmo.Model(source=modelo,effects=[dust],effect_names=['mw'],effect_frames=['obs'])
    model.set(mwebv=meta["MWEBV"],z=meta["REDSHIFT_FINAL"]) 
    try:
        result, fitted_model = sncosmo.fit_lc(dat, model, ['t0', 'x0', 'x1', 'x2'],modelcov=False, phase_range=(-10,40), wave_range=(3400,7000), verbose=False)
    except:
        print(f"SN {sn} fit returns NaN")
        result=None
        fitted_model=None
    
    return meta, surv, rm_f, dat, result, fitted_model

In [20]:
pp = PdfPages(f"./lc_fit_plot_{modelo.upper()}_SNLS_Pantheon.pdf")

In [21]:
colors1=["#440154"]
colors2=["#fde725","#440154"]
colors3=["#fde725","#21918c","#440154"]
colors4=["#fde725","#35b779","#31688e","#440154"]

In [22]:
 def return_fit_data(meta,res,model_name):

    return [meta["SNID"], "SNLS", meta["REDSHIFT_FINAL"], 0., res.parameters[1],
           res.errors["t0"], res.parameters[2], res.errors["x0"], res.parameters[3],
           res.errors["x1"], res.parameters[4], res.errors["x2"],
           res.covariance[1][2], res.covariance[1][3], res.covariance[2][3], 
           res.chisq/res.ndof, model_name]

In [23]:
def lc_fit_plot(sn, surv,table_data, result, fitted_model):
    z=round(result["parameters"][0],2)
    t0=round(result["parameters"][1],2)
    x1=round(result["parameters"][3],2)
    x1err=round(result["errors"]["x1"],2)
    x2=round(result["parameters"][4],2)
    x2err=round(result["errors"]["x2"],2)
    mwebv=result["parameters"][5]
    phase=np.linspace(t0-10.,t0+50.,100)

    if result.ndof>0:
        fil=[]
        for i,f in enumerate(magsystem.keys()): 
            if len(table_data[table_data["FILTER"]==f])==0:
                continue
            else:
                try:
                    fitted_model.bandflux(f, phase)
                    fil.append(f)
                    continue
                except:
                    continue
        fig, axs=plt.subplots(len(fil),1,figsize=(10,4*len(fil)),sharex=True)
        fig.subplots_adjust(hspace=0.1)
        if len(fil)==1:
            for i,f in enumerate(fil): 
                colors=eval("colors"+str(len(fil)))
                axs.plot(phase-t0,fitted_model.bandflux(f, phase)*10**(-0.4*table_data[table_data["FILTER"]==f]["ZP"][0]+11),linewidth=3,color=colors[i],label=f"{modelo.upper()} fit ($\chi^2/ndof$={round(result.chisq/result.ndof,2)})")
                axs.errorbar(table_data[table_data["FILTER"]==f]["MJD"]-t0,table_data[table_data["FILTER"]==f]["FLUXCAL"],yerr=table_data[table_data["FILTER"]==f]["FLUXCALERR"],label=f"Flux through {f} filter",ls="none", marker='o',markersize=8,color=colors[i])
                axs.set_ylabel(r"FLUX ($Z_{AB}$=27.5)")
                axs.set_xlim(-12,52)
                axs.set_title(f"{sn}({surv}),   z:{z:.2f},   x1={x1:.2f}$\pm${x1err:.2f},   x2={x2:.2f}$\pm${x2err:.2f}")
                formatter11=ScalarFormatterClass1()
                formatter11.set_scientific(True)
                formatter11.set_powerlimits((0,0))
                axs.yaxis.set_major_formatter(formatter11)
                axs.legend(loc=3)
                if i==len(fil)-1:
                    axs.set_xlabel(f"MJD-{t0}")
            #plt.show()
            plt.tight_layout()
            pp.savefig()
            plt.close(fig)
        else:
            for i,f in enumerate(fil): 
                colors=eval("colors"+str(len(fil)))
                axs[i].plot(phase-t0,fitted_model.bandflux(f, phase)*10**(-0.4*table_data[table_data["FILTER"]==f]["ZP"][0]+11),linewidth=3,color=colors[i],label=f"{modelo.upper()} fit ($\chi^2/ndof$={round(result.chisq/result.ndof,2)})")
                axs[i].errorbar(table_data[table_data["FILTER"]==f]["MJD"]-t0,table_data[table_data["FILTER"]==f]["FLUXCAL"],yerr=table_data[table_data["FILTER"]==f]["FLUXCALERR"],label=f"Flux through {f} filter",ls="none", marker='o',markersize=8,color=colors[i])
                axs[i].set_ylabel(r"FLUX ($Z_{AB}$=27.5)")
                axs[i].set_xlim(-12,52)
                axs[i].set_title(f"{sn}({surv}),   z:{z:.2f},   x1={x1:.2f}$\pm${x1err:.2f},   x2={x2:.2f}$\pm${x2err:.2f}")
                formatter11=ScalarFormatterClass1()
                formatter11.set_scientific(True)
                formatter11.set_powerlimits((0,0))
                axs[i].yaxis.set_major_formatter(formatter11)
                axs[i].legend(loc=3)
                if i==len(fil)-1:
                    axs[i].set_xlabel(f"MJD-{t0}")
            #plt.show()
            plt.tight_layout()
            pp.savefig()
            plt.close(fig)
    else:
        pass

In [24]:
all_fit_data=[]
f=open(f"./lc_failed_fits_{modelo.upper()}_SNLS_Pantheon.txt","w")
for j,sn in enumerate(sne_list):
    print(sn, f"({j}/{len(sne_list)})")
    meta, surv, rm_f, table_data, result, fitted_model=lc_fit(sn) 

    if result is None:
        f.write(f"SN {sn} fit returns NaN\n")
        continue
    else:
        if result.success and result.ndof>0:
            all_fit_data.append(return_fit_data(meta,result,f"{modelo.upper()}"))
        elif result.success and result.ndof==0:
            f.write("ndof 0 for SN"+sn+"\n")
        elif not result.success:
            f.write("Unsuccessful fit of SN"+sn+"\n")

    lc_fit_plot(sn, surv, table_data, result, fitted_model)                       

03D1au (0/236)


/home/cassia/anaconda3/envs/sne/lib/python3.10/site-packages/sncosmo/fitting.py:187: RuntimeWarning: Dropping following bands from data: SNLS_g(out of model wavelength range)
  warnings.warn("Dropping following bands from data: " +
/home/cassia/anaconda3/envs/sne/lib/python3.10/site-packages/iminuit/util.py:143: UserWarning: Assigned errors must be positive. Non-positive values are replaced by a heuristic.
  warnings.warn(


03D1ax (1/236)


/home/cassia/anaconda3/envs/sne/lib/python3.10/site-packages/sncosmo/fitting.py:187: RuntimeWarning: Dropping following bands from data: SNLS_g(out of model wavelength range)
  warnings.warn("Dropping following bands from data: " +
/home/cassia/anaconda3/envs/sne/lib/python3.10/site-packages/iminuit/util.py:143: UserWarning: Assigned errors must be positive. Non-positive values are replaced by a heuristic.
  warnings.warn(


03D1co (2/236)


...

/home/cassia/anaconda3/envs/sne/lib/python3.10/site-packages/sncosmo/fitting.py:187: RuntimeWarning: Dropping following bands from data: SNLS_g, SNLS_r(out of model wavelength range)
  warnings.warn("Dropping following bands from data: " +
/home/cassia/anaconda3/envs/sne/lib/python3.10/site-packages/iminuit/util.py:143: UserWarning: Assigned errors must be positive. Non-positive values are replaced by a heuristic.
  warnings.warn(


06D4cl (234/236)


/home/cassia/anaconda3/envs/sne/lib/python3.10/site-packages/sncosmo/fitting.py:187: RuntimeWarning: Dropping following bands from data: SNLS_r(out of model wavelength range)
  warnings.warn("Dropping following bands from data: " +
/home/cassia/anaconda3/envs/sne/lib/python3.10/site-packages/iminuit/util.py:143: UserWarning: Assigned errors must be positive. Non-positive values are replaced by a heuristic.
  warnings.warn(


06D4cq (235/236)


/home/cassia/anaconda3/envs/sne/lib/python3.10/site-packages/sncosmo/fitting.py:187: RuntimeWarning: Dropping following bands from data: SNLS_g(out of model wavelength range)
  warnings.warn("Dropping following bands from data: " +
/home/cassia/anaconda3/envs/sne/lib/python3.10/site-packages/iminuit/util.py:143: UserWarning: Assigned errors must be positive. Non-positive values are replaced by a heuristic.
  warnings.warn(


In [25]:
pp.close()

In [26]:
f.close()

## Salvando dados dos ajustes

In [27]:
df=pd.DataFrame(all_fit_data)

In [28]:
df=pd.DataFrame(all_fit_data)
df.columns=("SNNAME", "SURVEY", "Z", "ZERR", "T0", "T0ERR", "X0", "X0ERR", "X1", "X1ERR", "X2/C", "X2/CERR", "COV_X0_X1", "COV_X0_X2/C", "COV_X1_X2/C", "CHISQ/NDOF", "MODEL")

In [29]:
df["mB"]=-2.5*np.log10(df["X0"])
df["mBERR"]=np.abs(-2.5*np.log10(np.e)/df["X0"]*df["X0ERR"])
df["COV_mB_X1"]=-2.5*np.log10(np.e)/df["X0"]*df["COV_X0_X1"]
df["COV_mB_X2/C"]=-2.5*np.log10(np.e)/df["X0"]*df["COV_X0_X2/C"]

In [30]:
df.to_csv(f"./lc_fit_results_{modelo.upper()}_SNLS_Pantheon.txt", sep=' ', index=False)